# ☁️ AWS Cloud Practitioner Essentials: S3, EC2, IAM

**Generated:** 2025-09-22

This notebook guides you through a focused, hands‑on session covering **Amazon S3**, **Amazon EC2**, and **AWS IAM**. It’s designed for the AWS Cloud Practitioner level, with practical steps you can keep in your portfolio.


## 🎯 Objectives
- Understand the purpose of **S3 (object storage)**, **EC2 (compute)**, and **IAM (identity & access)**.
- Create and configure an **S3 bucket**, launch a **t2.micro EC2 instance**, and set basic **IAM** permissions.
- Wire them together (EC2 ↔ S3 with IAM permissions) and document the workflow.


## 🔗 Core Resources
- **AWS Training — Cloud Practitioner Essentials (free):** https://explore.skillbuilder.aws/learn/course/external/view/elearning/134/aws-cloud-practitioner-essentials
- **Amazon S3 docs:** https://docs.aws.amazon.com/s3/index.html
- **Amazon EC2 docs:** https://docs.aws.amazon.com/ec2/index.html
- **AWS IAM docs:** https://docs.aws.amazon.com/iam/index.html
- **AWS Free Tier:** https://aws.amazon.com/free/
- **AWS CLI install:** https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html


## ✅ Prerequisites
- An AWS account (Free Tier eligible).
- **AWS CLI** installed and configured (`aws configure`).  
- A key pair for EC2 SSH (or be ready to create one in the console).
- Basic terminal access (macOS/Linux Terminal, or Windows PowerShell).

---
## 1) Amazon S3 — Create and Configure a Bucket
**Goal:** Create a bucket, upload a sample file, enable versioning, and explore storage classes.


### Steps (Console)
1. Go to the **S3 console** → *Create bucket*.
2. Choose a **globally unique** bucket name (e.g., `mlops-finance-elvin-<random>`), select a region, keep defaults (Block Public Access **ON**).
3. **Create bucket**.
4. Upload a small CSV (e.g., `prices.csv`) to the bucket.
5. Enable **Versioning** in *Properties* and upload the file again to see versions.
6. Explore **Storage class** options (e.g., Standard vs. Infrequent Access).

> 📸 **Insert screenshot(s) below:**

![S3 Bucket Screenshot](../assets/s3_screenshot.png)

### (Optional) Steps via AWS CLI
```bash
# create bucket (replace REGION and BUCKET)
aws s3api create-bucket --bucket <YOUR_BUCKET> --region <REGION> --create-bucket-configuration LocationConstraint=<REGION>

# enable versioning
aws s3api put-bucket-versioning --bucket <YOUR_BUCKET> --versioning-configuration Status=Enabled

# upload file
aws s3 cp prices.csv s3://<YOUR_BUCKET>/data/prices.csv

# list objects
aws s3 ls s3://mlops-finance-data/data/
```


---
## 2) Amazon EC2 — Launch and Connect
**Goal:** Launch a **t2.micro** (Free Tier) Amazon Linux instance and SSH into it.


### Steps (Console)
1. Go to **EC2 console** → *Launch instance*.
2. Name it: `mlops-finance-ec2`.
3. AMI: **Amazon Linux 2023** (or Amazon Linux 2).
4. Instance type: **t2.micro** (Free Tier eligible).
5. Key pair: create or select an existing one.
6. Security group: allow **SSH (22)** from *My IP*.
7. Launch, wait for **running**, then copy the **Public IPv4 DNS**.
8. SSH from your terminal:
```bash
chmod 400 <your-key>.pem
ssh -i <your-key>.pem ec2-user@<EC2_PUBLIC_DNS>
```
9. On the instance, make sure Python is available (e.g., `python3 --version`).

> 📸 **Insert screenshot(s) below:**

![EC2 Instance Screenshot Placeholder](attachment:ec2_screenshot.png)

### (Optional) S3 from EC2
If you attach an **IAM role** with S3 permissions to the instance, you can:
```bash
# On EC2 (Amazon Linux), install aws cli v2 if not present
sudo dnf install -y awscli

# Copy a file to S3 (no access keys needed when instance role is attached)
echo "hello" > hello.txt
aws s3 cp hello.txt s3://<YOUR_BUCKET>/hello.txt
```


---
## 3) AWS IAM — Users, Policies, Roles
**Goal:** Create a **read‑only S3 user**, generate access keys (for local CLI), and create an **instance role** for EC2 to access S3.


### Steps (Console)
**User (Read‑only S3):**
1. IAM console → *Users* → *Create user* → `s3-readonly-user`.
2. *Attach policies directly* → add `AmazonS3ReadOnlyAccess`.
3. Create **access key** (programmatic). Store them securely.

**Instance Role (EC2 ↔ S3):**
1. IAM console → *Roles* → *Create role* → **Trusted entity: EC2**.
2. Attach policy `AmazonS3FullAccess` (or least-privilege custom policy for your bucket).
3. Name it `ec2-s3-access-role` and create.
4. Go to EC2 instance → *Actions* → *Security* → *Modify IAM role* → attach `ec2-s3-access-role`.

> 📸 **Insert screenshot(s) below:**

![IAM Screenshot Placeholder](attachment:iam_screenshot.png)

### (Optional) Verify permissions
```bash
# Locally with read‑only user (after aws configure)
aws s3 ls
aws s3 cp somefile.txt s3://<YOUR_BUCKET>/   # expected to FAIL (read-only)

# On EC2 with instance role
aws s3 cp hello.txt s3://<YOUR_BUCKET>/      # expected to SUCCEED
```


---
## 4) Wire It Together — Mini Flow
**Scenario:** You generate a CSV of model outputs on EC2 and push it to S3 with the EC2 instance role.

```bash
# On EC2
python3 - << 'PY'
import csv, random
from datetime import date
rows = [('date','risk_score')]
for i in range(5):
    rows.append((str(date.today()), round(random.random(), 4)))
with open('risk_scores.csv','w') as f:
    csv.writer(f).writerows(rows)
print("wrote risk_scores.csv")
PY

aws s3 cp risk_scores.csv s3://<YOUR_BUCKET>/exports/risk_scores.csv
```


## 🔒 Security Best Practices (Quick)
- Prefer **IAM roles** over long‑lived access keys on servers.
- Keep **S3 buckets private**; use signed URLs or CloudFront if public access is needed.
- Follow **least privilege**: restrict policies to specific buckets/paths.
- Rotate credentials; never commit keys to GitHub.


---
## 🧠 Reflection (fill in)
- What did S3, EC2, and IAM each contribute to this workflow?
- Where would you enforce least privilege?
- How would you integrate this into an **MLOps** pipeline (e.g., training artifacts → S3, batch inference on EC2/Lambda)?


## ✅ Checklist
- [ ] S3 bucket created, versioning enabled, file uploaded
- [ ] EC2 instance launched and accessible via SSH
- [ ] IAM user with S3 read‑only tested from local CLI
- [ ] IAM role attached to EC2 and S3 upload verified
- [ ] Mini flow (CSV → S3) completed
- [ ] Screenshots added to this notebook
